In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

In [15]:
df=pd.read_csv(r"C:\Users\ASUS\Desktop\sms.txt",sep='\t',names=["Result","Message"])

In [16]:
for i in range(len(df)):
    words=nltk.word_tokenize(df['Message'][i])
    words = re.sub('[^a-zA-Z]', ' ', df['Message'][i])
    words=words.lower()
    words=words.split(" ")
    words=[stemmer.stem(word) for word in words if word not in stopwords.words("english")]
    df['Message'][i]="  ".join(words)
df


,Result,Message
0,ham,go jurong point crazi avail bugi n...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup fin...
3,ham,u dun say earli hor u c alreadi ...
4,ham,nah think goe usf live around though
...,...,...
5567,spam,nd time tri contact u u ...
5568,ham,b go esplanad fr home
5569,ham,piti mood suggest
5570,ham,guy bitch act like interest buy someth ...


In [18]:
X=df.iloc[:,-1]

In [19]:
Y=pd.get_dummies(df['Result'],drop_first=True)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)

In [21]:
X=cv.fit_transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
Y.shape

(5572, 1)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [24]:
import tensorflow as tf
from tensorflow import keras

In [26]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024,activation='relu',input_shape=(3000,)))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [27]:
model.compile(optimizer='Adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train, epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4457 samples
Epoch 1/10
4457/4457 [==============================] - 5s 1ms/sample - loss: 0.2415 - accuracy: 0.9201
Epoch 2/10
4457/4457 [==============================] - 5s 1ms/sample - loss: 0.0518 - accuracy: 0.9852
Epoch 3/10
4457/4457 [==============================] - 4s 969us/sample - loss: 0.0291 - accuracy: 0.9928 - loss: 0.0294 - accuracy
Epoch 4/10
4457/4457 [==============================] - 5s 1ms/sample - loss: 0.0190 - accuracy: 0.9946
Epoch 5/10
4457/4457 [==============================] - 4s 982us/sample - loss: 0.0134 - accuracy: 0.9964
Epoch 6/10
4457/4457 [==============================] - 5s 1ms/sample - loss: 0.0096 - accuracy: 0.9978
Epoch 7/10
4457/4457 [==============================] - 5s 1ms/sample - loss: 0.0074 - accuracy: 0.9980
Epoch 8/10
4457/4457 [==============================] - 4s 1ms/sample - loss: 0.0057 - accuracy: 0.9982
Epoch 9/10
4457/4457 [

In [30]:
model.evaluate(X_test, y_test, verbose=2)

1115/1115 - 0s - loss: 0.0642 - accuracy: 0.9892


[0.06415432039063608, 0.98923767]

In [61]:
predictions = model.predict(X_test)

In [62]:
b=list(predictions)
for i in range(len(b)):
    if b[i]>0.5:
        b[i]=1
    else:
        b[i]=0


In [63]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,b)

array([[954,   1],
       [ 11, 149]], dtype=int64)

In [64]:
from sklearn.metrics import classification_report
print(classification_report(y_test, b))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       955
           1       0.99      0.93      0.96       160

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

